In [ ]:
using HDF5, EzXML, Glob, Dates, Plots, Optim, LineSearches, LsqFit, JuMP
import NLopt
include("/Users/jjc/CSF/CSF Dynamics Scripts/readCSF.jl")

datapath = "/Users/jjc/Documents/SSC/CSF Data/selected_recordings"
path = pwd();
savepath = "/Users/jjc/CSF/"
files = glob("*.hdf5", datapath);

# filename =  datapath*"/INF_20200909122333_INF2.hdf5"
# filename = datapath*"/INF_20190708150215_INF2.hdf5"
# filename = datapath*"/INF_20191010113919_INF2.hdf5"
filename = datapath * "/INF_9CA2CCEC_E238_4CB0_99DC_78B94C06CAB9.hdf5"
# filename = datapath*"/INF_20190620142837_INF2.hdf5"
# filename = datapath * "/inf_20180329121402.hdf5"
# filename = datapath * "/INF_20180829151805_INF2.hdf5"

Data = readCSF(filename)

T = Data["T"]
rec_dur_s = Data["rec_dur_s"]
start_time = Data["start_time"]
end_time = Data["end_time"]
infusion_start_frame = Data["infusion_start_frame"]
infusion_end_frame = Data["infusion_end_frame"]
plateau_start = Data["plateau_start"]
plateau_end = Data["plateau_end"]

ICP = Data["ICP"]

E = Data["E"]
P_0 = Data["P_0"]
P_p = Data["P_p"]
P_b = Data["P_b"]
I_b = Data["I_b"]
I_inf = Data["I_inf"]
;

In [1]:
function myerrfun(I_b::Real, E::Real, P_0::Real)
    err = 0.0
    for i = 1:length(Pm)
        tᵢ = (i - 1) / 6
        global t = tᵢ
        It = I_b + I_inf
        ΔP = P_b - P_0
        predᵢ = It * ΔP / (I_b + (I_inf * exp(-E * It * tᵢ))) + P_0
        MODVAL[i] = predᵢ
        err += (Pm[i] - predᵢ)^2
    end

    # X = [value(I_b), value(E), value(P_0)]
    # δΔP = maximum([0.0, X[3] - P_b])
    # δlb = delta.(lower .- X)
    # δub = delta.(X .- upper)
    # δ = C .* vcat(δlb, δub, δΔP) # i = 1,⋯,q
    # # Distances do not make 100% because it will force the optimum to be in the middle lower/upper
    # dist = vcat(lower .- X, X .- upper) # Distances between the constraint and the current value
    # h = C .* abs.(dist) # i = q+1,⋯,m
    # d = vcat(δ, h)
    # penalty = sum(d .^ κ)
    # # penalty = sum(δ .^ κ)
    # err += penalty
    return err
end

myerrfun (generic function with 1 method)

In [ ]:
Pm = ICP[infusion_start_frame:infusion_end_frame]
t = 0.0
MODVAL = zeros(length(Pm))

dsampf = 20
numsamples = length(ICP)
gg = moving_average(ICP, dsampf)
g0 = zeros(length(ICP))
g0 .+= P_b
g0[Int(dsampf / 2):Int(dsampf / 2)+length(gg)-1] = gg
g0[Int(dsampf / 2)+length(gg):end] .= P_p

# SET PARAMS FOR OPTIM
# Pm = ICP[infusion_start_frame:infusion_end_frame]
Pm = g0[infusion_start_frame:infusion_end_frame]
lower = [0.0, 0.0, -5.0]
upper = [1.0, 1.0, P_b]
C = 10^3
κ = 2
# x0 = [Ibest, Eest, P0est]
x0 = [0.3, 0.16, 10.0]
alglist = [:LN_NELDERMEAD, :GN_ISRES, :GN_DIRECT, :GN_DIRECT_L, :GN_DIRECT_L_RAND, :GN_CRS2_LM]
# alglist = [:GN_DIRECT_L]

minerrval = 10.0
minerrmet = ""
for optimMethod in alglist
    optalg = optimMethod
    model, I_b, E, P_0 = getModel(optalg, x0)
    errval = objective_value(model)
    if I_b < minerrval && I_b > 0.15
        global minerrval = I_b
        global minmet = optimMethod
    end
    println("Method: $optimMethod\nError value:\n$errval")
    println("Estimated parameters:\nIₐ = $I_b [mL/min]\n" * "E = $E [mmHg/mL]\n" * "P₀ = $P_0 [mmHg]\n")
end
# Pm = g0[infusion_start_frame:infusion_end_frame]
plot_model(I_b, E, P_0, ICP, dsampf)
# minerrval, minmet